In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [35]:
train_dataset,target=load_script.load_dataset(train=True,single_target=True)

In [36]:
test_dataset,test_target=load_script.load_dataset(train=False,single_target=True)

In [37]:
train_svm=train_dataset.data.view(316,-1).numpy()

In [38]:
target_svm=target.data.numpy()

In [39]:
test_svm=test_dataset.data.view(100,-1).numpy()

In [40]:
test_target_svm=test_target.data.numpy()

# Try SVM

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
clf = RandomForestClassifier()

In [44]:
train_svm=train_svm[:,:50]
test_svm=test_svm[:,:50]

In [45]:
clf.fit(train_svm, target_svm)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
out=clf.predict(test_svm)

In [47]:
np.sum(test_target_svm==out)

60

# Defining the network

### Single output

In [6]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

# Training

In [7]:
net=SingleOutput()

optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion=nn.MSELoss()
mini_batch_size=100

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        
#         print(train_element.shape)
        out=net(train_element)
#         print(out)
        output_target[b:b+mini_batch_size]=(out>0.5).data
#         print(out)
#         print(target_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
#         print(out)
    output_test=(out>0.5).data.type(torch.FloatTensor)
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target==target.data))/target.shape[0]
    error_test=np.sum(list(output_test[:,0]==test_target.data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

RuntimeError: invalid argument 4: out of range at /Users/soumith/minicondabuild3/conda-bld/pytorch_1518385717421/work/torch/lib/TH/generic/THTensor.c:440